In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from itertools import combinations, chain

from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc

from joblib import dump

In [2]:
df = pd.read_csv('../data/balanced_sclaer_dataset_diabetes.csv')

In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
df.isnull().sum().sum()

0

In [5]:
y = df["Diabetes_012"]
X = df.drop(["Diabetes_012"], axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [7]:
cbc = CatBoostClassifier(learning_rate=0.05, 
                        task_type="GPU",
                        devices='0:4',
                        logging_level='Verbose'
                        )

erbose — Output the following data to stdout:

* optimized metric
* elapsed time of training
* remaining time of training

In [8]:
params = {'depth': np.arange(3, 10), 
          'l2_leaf_reg': np.arange(1, 20),
          'bootstrap_type': ['Bayesian', 'Bernoulli', 'Poisson'],
          'num_trees': np.arange(1, 20)
          }

In [9]:
catboost_classifier = cbc.randomized_search(params, X=X_train[0:1000], y=y_train[0:1000], n_iter=3)

0:	learn: 1.0851707	test: 1.0859483	best: 1.0859483 (0)	total: 9.77ms	remaining: 58.6ms
1:	learn: 1.0737978	test: 1.0751582	best: 1.0751582 (1)	total: 17.8ms	remaining: 44.5ms
2:	learn: 1.0621120	test: 1.0641824	best: 1.0641824 (2)	total: 25.3ms	remaining: 33.8ms
3:	learn: 1.0502919	test: 1.0539333	best: 1.0539333 (3)	total: 33ms	remaining: 24.8ms
4:	learn: 1.0364744	test: 1.0396290	best: 1.0396290 (4)	total: 41.5ms	remaining: 16.6ms
5:	learn: 1.0266420	test: 1.0299426	best: 1.0299426 (5)	total: 50ms	remaining: 8.33ms
6:	learn: 1.0139904	test: 1.0168272	best: 1.0168272 (6)	total: 58ms	remaining: 0us
bestTest = 1.016827164
bestIteration = 6
0:	loss: 1.0168272	best: 1.0168272 (0)	total: 291ms	remaining: 582ms
0:	learn: 1.0775548	test: 1.0776524	best: 1.0776524 (0)	total: 13.9ms	remaining: 55.6ms
1:	learn: 1.0570544	test: 1.0599390	best: 1.0599390 (1)	total: 26.9ms	remaining: 40.3ms
2:	learn: 1.0368778	test: 1.0409631	best: 1.0409631 (2)	total: 39ms	remaining: 26ms
3:	learn: 1.0167662	tes

In [12]:
cbc.fit(X_train, y_train)

0:	learn: 1.0610596	total: 51.6ms	remaining: 206ms
1:	learn: 1.0280341	total: 76.1ms	remaining: 114ms
2:	learn: 0.9966855	total: 93.6ms	remaining: 62.4ms
3:	learn: 0.9705781	total: 112ms	remaining: 28.1ms
4:	learn: 0.9443678	total: 131ms	remaining: 0us


In [13]:
cbc_predict = cbc.predict(X_test)
cbc_predict

array([[2.],
       [1.],
       [2.],
       ...,
       [1.],
       [2.],
       [2.]])

In [14]:
print(classification_report(y_test, cbc_predict))

              precision    recall  f1-score   support

         0.0       0.69      0.67      0.68     11774
         1.0       0.98      0.69      0.81     11638
         2.0       0.58      0.77      0.67     11685

    accuracy                           0.71     35097
   macro avg       0.75      0.71      0.72     35097
weighted avg       0.75      0.71      0.72     35097

